In [1]:
from attentional_cpmp.utils.hyperparameter_search import HyperparameterStudy
from attentional_cpmp.utils.data_saving.data_json import load_data_from_json

c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random

def split_data(data, key, percentage):
    # Obtener el diccionario para la clave específica
    subset = data.get(key)
    if not subset:
        raise KeyError(f"No existe la clave {key} en el diccionario.")

    states = subset["States"]
    labels = subset["Labels"]

    # Determinar la cantidad de elementos a extraer
    sample_size = int(len(states) * percentage)

    # Obtener índices aleatorios para la muestra
    indices = random.sample(range(len(states)), sample_size)

    # Crear subconjuntos para el 20% y el 80%
    sampled_states = [states[i] for i in indices]
    sampled_labels = [labels[i] for i in indices]

    remaining_states = [states[i] for i in range(len(states)) if i not in indices]
    remaining_labels = [labels[i] for i in range(len(labels)) if i not in indices]

    return (sampled_states, sampled_labels), (remaining_states, remaining_labels)

In [3]:
data_Sx7 = load_data_from_json('CPMP_With_Attention.Sx7_v4.json')

In [4]:
(val_x, val_y), (x, y) = split_data(data_Sx7, '5', 0.2)

In [5]:
len(val_x), len(x)

(12625, 50503)

In [6]:
study = HyperparameterStudy()

[I 2024-10-27 14:53:15,236] A new study created in memory with name: Study_Model_CPMP


In [7]:
study.set_training_data(X_train=x, Y_train=y,
                        X_val=val_x, Y_val=val_y,
                        epochs=10, batch_size=32)

study.set_max_config_trial()
study.set_config_model(H=7, n_dropout_hide=1, n_dropout_feed=2,
                       optimizer='Adam', loss='binary_crossentropy')
study.set_config_callbacks()

Directory 'hyperparameter_test/' already exists.


In [ ]:
study.optimize(n_trials=10)